# Battle of the Neighborhoods - Coursera Capstone Project

### This Notebook contains the data analysis to assist the client in selecting ideal location(s) for their new concept in quick service health food in New York City.

#### Import all required Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import csv # implements classes to read and write tabular data in CSV form

print('All Set!!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

### Download and read NYC json file

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('NYC json file has been downloaded')

NYC json file has been downloaded


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Import Data into Pandas Dataframe

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


In [9]:
neighborhoods.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


#### Make sure that all 5 Boroughs and 306 Neighborhoods were downloaded

In [10]:
print('There are {} boroughs and {} neighborhoods in the dataset.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

There are 5 boroughs and 306 neighborhoods in the dataset.


In [11]:
neighborhoods.to_csv('NYC_neigh_GEO.csv',index=False)

#### Obtain coordinates for NYC using geopy

In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create map of NYC and show neighborhoods

In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork



#### Import more libraries

In [14]:
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup # package for parsing HTML and XML documents

print('All Set!!')

All Set!!


### Import population data for NYC from wikipedia and scrape data using Beautiful Soup

###### location of data source:  https://en.wikipedia.org/wiki/Demographics_of_New_York_City

In [15]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())


headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('NYC_POP.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

#### Load and clean data

In [16]:
Pop_data=pd.read_csv('NYC_POP.csv')
Pop_data.drop(Pop_data.columns[[1,3,4,6,8,9,10,11,12,13,14]], axis=1,inplace=True)
print('NYC population data downloaded!')

NYC population data downloaded!


In [17]:
Pop_data.columns = Pop_data.columns.str.replace(' ', '')
Pop_data.columns = Pop_data.columns.str.replace('\'','')
#Pop_data.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
Pop_data

,NewYorkCitysfiveboroughsvte,Population,Density,County
0,The Bronx\n,"1,418,207\n",42.10\n,"33,867\n"
1,Brooklyn\n,"2,559,903\n",70.82\n,"36,147\n"
2,Manhattan\n,"1,628,706\n",22.83\n,"71,341\n"
3,Queens\n,"2,253,858\n",108.53\n,"20,767\n"
4,Staten Island\n,"476,143\n",58.37\n,"8,157\n"
5,City of New York,842.343,783.83,"10,636\n"
6,State of New York,"1,731.910","122,284",159\n
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN


In [18]:
Pop_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'Borough',        
                   'Population\n':'Pop_2019', 
                   'Density\n': 'sq_miles',
                    'County': 'persons_sq_mi'}, inplace=True)
Pop_data


,Borough,Pop_2019,sq_miles,persons_sq_mi
0,The Bronx\n,"1,418,207\n",42.10\n,"33,867\n"
1,Brooklyn\n,"2,559,903\n",70.82\n,"36,147\n"
2,Manhattan\n,"1,628,706\n",22.83\n,"71,341\n"
3,Queens\n,"2,253,858\n",108.53\n,"20,767\n"
4,Staten Island\n,"476,143\n",58.37\n,"8,157\n"
5,City of New York,842.343,783.83,"10,636\n"
6,State of New York,"1,731.910","122,284",159\n
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN


In [19]:
Pop_data.drop([5,6,7], axis=0,inplace=True)
Pop_data

,Borough,Pop_2019,sq_miles,persons_sq_mi
0,The Bronx\n,"1,418,207\n",42.10\n,"33,867\n"
1,Brooklyn\n,"2,559,903\n",70.82\n,"36,147\n"
2,Manhattan\n,"1,628,706\n",22.83\n,"71,341\n"
3,Queens\n,"2,253,858\n",108.53\n,"20,767\n"
4,Staten Island\n,"476,143\n",58.37\n,"8,157\n"


In [20]:
Pop_data['Borough']=Pop_data['Borough'].replace(to_replace='\n', value='', regex=True)
Pop_data['Pop_2019']=Pop_data['Pop_2019'].replace(to_replace='\n', value='', regex=True)
Pop_data['sq_miles']=Pop_data['sq_miles'].replace(to_replace='\n', value='', regex=True)
Pop_data['persons_sq_mi']=Pop_data['persons_sq_mi'].replace(to_replace='\n', value='', regex=True)

Pop_data

,Borough,Pop_2019,sq_miles,persons_sq_mi
0,The Bronx,"1,418,207",42.10,"33,867"
1,Brooklyn,"2,559,903",70.82,"36,147"
2,Manhattan,"1,628,706",22.83,"71,341"
3,Queens,"2,253,858",108.53,"20,767"
4,Staten Island,"476,143",58.37,"8,157"


In [21]:
Pop_data.to_csv('NYC_POP_final.csv',index=False)

### Read in Crime data for NYC 

##### crime data taken from kaggle at: https://www.kaggle.com/adamschroeder/crimes-new-york-city

In [22]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e628b851e2e148efa4ae0d884b4e2d73 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Eub9fTPdYysDWEjYUC-HGnDrSB-e6nmuMQTcv8ZlCT6w',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_e628b851e2e148efa4ae0d884b4e2d73.get_object(Bucket='courseracapstone-donotdelete-pr-gazuqcaxq2jqrt',Key='NYC_felonies.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,CMPLNT_NUM,CMPLNT_FR_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,JURIS_DESC,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,531559484,1/1/2015,109,GRAND LARCENY,407.0,"LARCENY,GRAND BY DISHONEST EMP",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,46,INSIDE,COMMERCIAL BUILDING,NaN,NaN,1006340,249963,40.852746,-73.920152,"(40.85274625, -73.920151953)"
1,370877689,1/1/2015,126,MISCELLANEOUS PENAL LAW,461.0,UNAUTHORIZED USE VEHICLE 2,COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,47,FRONT OF,STREET,NaN,NaN,1022541,259736,40.879515,-73.861534,"(40.879514742, -73.861533865)"
2,305625874,1/1/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",COMPLETED,FELONY,N.Y. TRANSIT POLICE,BRONX,46,NaN,TRANSIT - NYC SUBWAY,NaN,NaN,1010257,248767,40.849453,-73.905998,"(40.849452916, -73.905997684)"
3,741596269,1/1/2015,109,GRAND LARCENY,438.0,"LARCENY,GRAND FROM BUILDING (NON-RESIDENCE) UN...",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,40,INSIDE,STORAGE FACILITY,NaN,NaN,1009712,233550,40.807688,-73.908025,"(40.807688206, -73.908025367)"
4,259570021,1/1/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,41,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1012780,236449,40.815636,-73.896931,"(40.815635776, -73.896930554)"


##### Group felonies by Borough

In [23]:
df_data_1.groupby('BORO_NM').count()

,CMPLNT_NUM,CMPLNT_FR_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,JURIS_DESC,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
BORO_NM,,,,,,,,,,,,,,,,,,,
BRONX,28285,28285,28285,28276,28197,28197,28285,28285,28285,28285,21129,28124,349,1311,28285,28285,28285,28285,28285
BROOKLYN,45827,45827,45827,45796,45690,45690,45827,45827,45827,45827,35843,45590,354,2897,45827,45827,45827,45827,45827
MANHATTAN,33983,33983,33983,33980,33944,33944,33983,33983,33983,33983,27240,33673,496,1038,33983,33983,33983,33983,33983
QUEENS,30403,30403,30403,30402,30345,30345,30403,30403,30403,30403,23625,30266,264,398,30403,30403,30403,30403,30403
STATEN ISLAND,4951,4951,4951,4947,4938,4938,4951,4951,4951,4951,4241,4914,26,222,4951,4951,4951,4951,4951


###### The resulting dataframe lists all felonies created in a calendar year grouped by Borough.  The data for each entry includes the latitude and longitude of the location where the crime was perpetrated.  Due to the large volume of data (even for only one calendar year), plotting the data on the map provides no value.  The value in this tool will become apparent once the client narrows down a specific location to buy or lease.  The client can then search through this data for the crimes committed in a given radius from that location to get a sense of the crime in that area.

In [24]:
df_data_1.head(20)

,CMPLNT_NUM,CMPLNT_FR_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,JURIS_DESC,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,531559484,1/1/2015,109,GRAND LARCENY,407.0,"LARCENY,GRAND BY DISHONEST EMP",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,46,INSIDE,COMMERCIAL BUILDING,NaN,NaN,1006340,249963,40.852746,-73.920152,"(40.85274625, -73.920151953)"
1,370877689,1/1/2015,126,MISCELLANEOUS PENAL LAW,461.0,UNAUTHORIZED USE VEHICLE 2,COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,47,FRONT OF,STREET,NaN,NaN,1022541,259736,40.879515,-73.861534,"(40.879514742, -73.861533865)"
2,305625874,1/1/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",COMPLETED,FELONY,N.Y. TRANSIT POLICE,BRONX,46,NaN,TRANSIT - NYC SUBWAY,NaN,NaN,1010257,248767,40.849453,-73.905998,"(40.849452916, -73.905997684)"
3,741596269,1/1/2015,109,GRAND LARCENY,438.0,"LARCENY,GRAND FROM BUILDING (NON-RESIDENCE) UN...",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,40,INSIDE,STORAGE FACILITY,NaN,NaN,1009712,233550,40.807688,-73.908025,"(40.807688206, -73.908025367)"
4,259570021,1/1/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,41,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1012780,236449,40.815636,-73.896931,"(40.815635776, -73.896930554)"
5,467833507,1/1/2015,109,GRAND LARCENY,403.0,"LARCENY,GRAND BY BANK ACCT COMPROMISE-REPRODUC...",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,41,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1013392,237231,40.817780,-73.894716,"(40.817780148, -73.894716176)"
6,768467551,1/1/2015,109,GRAND LARCENY,422.0,"LARCENY,GRAND BY OPEN CREDIT CARD (NEW ACCT)",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,52,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1017440,256045,40.869405,-73.879998,"(40.869404748, -73.879998318)"
7,598793688,1/1/2015,109,GRAND LARCENY,420.0,"LARCENY,GRAND BY OPEN/COMPROMISE CELL PHONE ACCT",COMPLETED,FELONY,N.Y. HOUSING POLICE,BRONX,42,INSIDE,RESIDENCE - PUBLIC HOUSING,NaN,NaN,1010585,242785,40.833033,-73.904836,"(40.833033106, -73.9048356)"
8,174641571,1/1/2015,110,GRAND LARCENY OF MOTOR VEHICLE,441.0,"LARCENY,GRAND OF AUTO",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,43,OPPOSITE OF,STREET,NaN,NaN,1023042,235210,40.812196,-73.859864,"(40.81219597, -73.859864038)"
9,613180643,1/1/2015,109,GRAND LARCENY,420.0,"LARCENY,GRAND BY OPEN/COMPROMISE CELL PHONE ACCT",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,45,INSIDE,TELECOMM. STORE,NaN,NaN,1030386,244420,40.837439,-73.833270,"(40.837439463, -73.833270322)"


###### Based on the demographics and the criteria put forth by the client, it is obvious that the intial locations should be somewhere in Manhattan.  The focus of the remainder of this analysis will be on the neighborhoods of Manhattan.

#### Segment and Cluster Neighborhoods

In [25]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

print('All Set!!')

All Set!!


In [26]:
NYC_Geo=pd.read_csv('NYC_neigh_GEO.csv')
print('Data downloaded!')

Data downloaded!


In [27]:
NYC_Geo.head(20)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [28]:
NYC_Geo['Borough'].value_counts().to_frame()

,Borough
Queens,81
Brooklyn,70
Staten Island,63
Bronx,52
Manhattan,40


In [29]:
NYC_Geo.shape

(306, 4)

In [30]:
print(NYC_Geo.Borough.unique())

['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']


In [31]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [32]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


### Create map of Manhattan

In [33]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Use Foursquare to explore the neighborhoods of Manhattan

In [34]:
CLIENT_ID = 'KEOT33Q5SXQIZ1AK2R5KNS12IUQIXU055HXV50LVQ5DE1VKZ' # your Foursquare ID
CLIENT_SECRET = 'QOBPOHUZTGPUFYBZELAZ0UG1PEPN4K2RZXYZLS0PT1NJXCRE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KEOT33Q5SXQIZ1AK2R5KNS12IUQIXU055HXV50LVQ5DE1VKZ
CLIENT_SECRET:QOBPOHUZTGPUFYBZELAZ0UG1PEPN4K2RZXYZLS0PT1NJXCRE


In [35]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues) 

In [37]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [38]:
print(manhattan_venues.shape)
manhattan_venues.head(20)

(3178, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
5,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
6,Marble Hill,40.876551,-73.91066,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium
7,Marble Hill,40.876551,-73.91066,Rite Aid,40.875467,-73.908906,Pharmacy
8,Marble Hill,40.876551,-73.91066,Starbucks,40.873755,-73.908613,Coffee Shop
9,Marble Hill,40.876551,-73.91066,Blink Fitness,40.877271,-73.905595,Gym


In [39]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,69,69,69,69,69,69
Carnegie Hill,92,92,92,92,92,92
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,42,42,42,42,42,42
East Village,100,100,100,100,100,100


In [40]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,South Indian 

In [41]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,South Indian 

In [42]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.10
1          Hotel  0.07
2            Gym  0.06
3    Coffee Shop  0.06
4  Boat or Ferry  0.04


----Carnegie Hill----
                venue  freq
0         Coffee Shop  0.08
1         Pizza Place  0.04
2                Café  0.04
3  Italian Restaurant  0.04
4         Yoga Studio  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1    French Restaurant  0.04
2  American Restaurant  0.04
3                  Bar  0.04
4   Seafood Restaurant  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.09
1          Art Gallery  0.06
2  American Restaurant  0.04
3    French Restaurant  0.03
4               Bakery  0.03


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.07
1              Bakery  0.05
2        Cocktail Bar  0.04
3        Dessert Shop  0.04
4   Hotpot Restaurant  0.03


----Civic Center----
                  venue  freq
0           

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Gym,Boat or Ferry,Memorial Site,Shopping Mall,Gourmet Shop,Plaza,BBQ Joint
1,Carnegie Hill,Coffee Shop,Café,Italian Restaurant,Pizza Place,Yoga Studio,Bar,Bakery,Gym,Gym / Fitness Center,Bookstore
2,Central Harlem,African Restaurant,Chinese Restaurant,American Restaurant,Bar,French Restaurant,Seafood Restaurant,Public Art,Bookstore,Boutique,Market
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,French Restaurant,Bakery,Ice Cream Shop,Pizza Place,Cycle Studio,Cocktail Bar,Italian Restaurant
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Dessert Shop,Vietnamese Restaurant,Bar,Ice Cream Shop,Hotpot Restaurant,Spa,Optical Shop


In [45]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 2, 1, 2, 2, 0, 1, 1], dtype=int32)

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Gym,Sandwich Place,Coffee Shop,Yoga Studio,Pizza Place,Big Box Store,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Bakery,Cocktail Bar,Dessert Shop,Vietnamese Restaurant,Bar,Ice Cream Shop,Hotpot Restaurant,Spa,Optical Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Café,Bakery,Deli / Bodega,Mobile Phone Shop,Chinese Restaurant,Bank,Latin American Restaurant,Tapas Restaurant,Supermarket,Park
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Lounge,Restaurant,Café,Bakery,Frozen Yogurt Shop,Caribbean Restaurant,Chinese Restaurant,Spanish Restaurant,Wine Bar
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Sandwich Place,Sushi Restaurant,Cocktail Bar,Bakery,Yoga Studio


In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Now a dataset called Relevant will be defined.  This dataset contains all of the venues that were agreed with the client would be locations that would like to be located near in order to attract the most foot traffic from their target audience.  These venue names are: Gym, Baseball Field, Basketball Court, Bike Rental/Bike Share, Bike Trail, Boxing Gym, Climbing Gym, Cycle Studio, Dance Studio, Gym/Fitness Center, Martial Arts Dojo, Massage Studio, Outdoors & Recreation, Park, Pool, Spa, Tennis Court, Trail, and Yoga Studio

In [48]:
Relevant = manhattan_venues.loc[manhattan_venues['Venue Category'].isin(['Gym','Baseball Field', 'Basketball Court', 'Bike Rental/Bike Share', 'Bike Trail', 'Boxing Gym', 'Climbing Gym', 'Cycle Studio', 'Dance Studio', 'Gym/Fitness Center', 'Martial Arts Dojo', 'Massage Studio', 'Outdoors & Recreation', 'Park', 'Pool', 'Spa', 'Tennis Court', 'Trail', 'Yoga Studio'])]


Relevant

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Marble Hill,40.876551,-73.910660,Bikram Yoga,40.876844,-73.906204,Yoga Studio
4,Marble Hill,40.876551,-73.910660,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
9,Marble Hill,40.876551,-73.910660,Blink Fitness,40.877271,-73.905595,Gym
36,Chinatown,40.715618,-73.994279,Renew Day Spa,40.715559,-73.996747,Spa
38,Chinatown,40.715618,-73.994279,GoGreen Organic Spa,40.717014,-73.995847,Spa
54,Chinatown,40.715618,-73.994279,Sky Ting Yoga,40.715352,-73.992666,Yoga Studio
84,Chinatown,40.715618,-73.994279,Zu Yuan Spa,40.715469,-73.998627,Spa
132,Washington Heights,40.851903,-73.936900,Highest Natural Point In Manhattan,40.852843,-73.937650,Park
137,Washington Heights,40.851903,-73.936900,Bennett Park,40.852967,-73.937874,Park
143,Washington Heights,40.851903,-73.936900,Highbridge Park Pool,40.849110,-73.936839,Pool


### Next a dataset called Competitor will be defined. This dataset contains all of the venues that were agreed with the client would be locations that would like to distance themselves from. These venue names are: Smoothie Shop, Health Food Store, Juice Bar, and Salad Place.

In [49]:
Competitor = manhattan_venues.loc[manhattan_venues['Venue Category'].isin(['Smoothie Shop', 'Health Food Store', 'Juice Bar', 'Salad Place'])]

Competitor

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
125,Chinatown,40.715618,-73.994279,Hawa Smoothies,40.714200,-73.989390,Juice Bar
261,Inwood,40.867684,-73.921210,disFruta,40.864613,-73.919199,Juice Bar
332,Manhattanville,40.816934,-73.957385,Oasis Juice Bar,40.815017,-73.958879,Juice Bar
384,Central Harlem,40.815976,-73.943211,Rejuvenate,40.813872,-73.944142,Juice Bar
485,Upper East Side,40.775639,-73.960508,CHOPT,40.773797,-73.957825,Salad Place
518,Upper East Side,40.775639,-73.960508,JOE & THE JUICE,40.775665,-73.958473,Juice Bar
522,Upper East Side,40.775639,-73.960508,Juice Generation,40.777423,-73.955242,Juice Bar
527,Upper East Side,40.775639,-73.960508,Juice Press,40.777150,-73.955289,Juice Bar
539,Upper East Side,40.775639,-73.960508,Pressed Juicery,40.778071,-73.956889,Juice Bar
541,Upper East Side,40.775639,-73.960508,Just Salad,40.777037,-73.955290,Salad Place


### Create a map af Manhattan with the "Relevant" locations shown as green dots.

In [50]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Relevant['Venue Latitude'], Relevant['Venue Longitude'], Relevant['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Create a map af Manhattan with the "Competitor" locations shown as red dots.

In [51]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Competitor['Venue Latitude'], Competitor['Venue Longitude'], Competitor['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan